## 4. Modeling 

#### Input: Cleansed Data File from Data_Understanding



In [3]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, precision_recall_curve, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

Read in data from csv file

In [4]:
employee_data = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv', sep=',')

In [5]:
employee_data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [6]:
#  Kategoriale Spalten identifizieren
categorical_columns = employee_data.select_dtypes(include=['object']).columns
print("Kategoriale Spalten:", categorical_columns) 

# Label Encoder initialisieren
label_encoder = LabelEncoder()

# Kopie des DataFrames erstellen, um die Originaldaten beizubehalten
employee_data_encoded = employee_data.copy()

# Label Encoder auf jede kategoriale Spalte anwenden
for column in categorical_columns:
    df_encoded[column] = label_encoder.fit_transform(employee_data_encoded[column])

# Die ersten Zeilen des kodierten DataFrames anzeigen
print(employee_data_encoded.head())

Kategoriale Spalten: Index(['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender',
       'JobRole', 'MaritalStatus', 'Over18', 'OverTime'],
      dtype='object')


NameError: name 'LabelEncoder' is not defined

In [ ]:
X = employee_data_encoded.drop('Attrition', axis=1)
y = employee_data.Attrition

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1,
                                                    stratify=y)

In [ ]:
stay = (y_train.value_counts()[0] / y_train.shape)[0]
leave = (y_train.value_counts()[1] / y_train.shape)[0]

print("===============TRAIN=================")
print(f"Staying Rate: {stay * 100:.2f}%")
print(f"Leaving Rate: {leave * 100 :.2f}%")

stay = (y_test.value_counts()[0] / y_test.shape)[0]
leave = (y_test.value_counts()[1] / y_test.shape)[0]

print("===============TEST=================")
print(f"Staying Rate: {stay * 100:.2f}%")
print(f"Leaving Rate: {leave * 100 :.2f}%")

===============TRAIN=================
Staying Rate: 83.85%
Leaving Rate: 16.15%
===============TEST=================
Staying Rate: 83.97%
Leaving Rate: 16.03%


C:\Users\vanes\AppData\Local\Temp\ipykernel_32428\329314148.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stay = (y_train.value_counts()[0] / y_train.shape)[0]
C:\Users\vanes\AppData\Local\Temp\ipykernel_32428\329314148.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  leave = (y_train.value_counts()[1] / y_train.shape)[0]
C:\Users\vanes\AppData\Local\Temp\ipykernel_32428\329314148.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc

In [ ]:

def evaluate(model, X_train, X_test, y_train, y_test):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    print("TRAINIG RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_train, y_train_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_train, y_train_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_train, y_train_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")

    print("TESTING RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_test, y_test_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_test, y_test_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_test, y_test_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")

In [ ]:
# Kategoriale und numerische Spalten identifizieren
categorical_columns = X.select_dtypes(include=['object']).columns
numeric_columns = X.select_dtypes(include=['number']).columns

# One-Hot-Encoder und StandardScaler initialisieren
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ])

# Pipeline erstellen
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# Pipeline auf die Trainingsdaten anpassen und transformieren
X_train_processed = pipeline.fit_transform(X_train)

# Pipeline auf die Testdaten anwenden
X_test_processed = pipeline.transform(X_test)

# Optional: Pipeline auf den gesamten Datensatz anwenden
X_processed = pipeline.transform(X)

#### SVM

In [ ]:
# Creation and training of the SVM-model
svm_clf = SVC(kernel='linear', C=1.0, random_state=1)
svm_clf.fit(X_train_processed, y_train)
evaluate(svm_clf, X_train_processed, X_test_processed, y_train, y_test)

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[919  14]
 [ 94  75]]
ACCURACY SCORE:
0.9020
CLASSIFICATION REPORT:
                   No         Yes  accuracy    macro avg  weighted avg
precision    0.907206    0.842697  0.901996     0.874951      0.897313
recall       0.984995    0.443787  0.901996     0.714391      0.901996
f1-score     0.944502    0.581395  0.901996     0.762948      0.888816
support    933.000000  169.000000  0.901996  1102.000000   1102.000000
TESTING RESULTS: 
CONFUSION MATRIX:
[[291   9]
 [ 44  24]]
ACCURACY SCORE:
0.8560
CLASSIFICATION REPORT:
                   No        Yes  accuracy   macro avg  weighted avg
precision    0.868657   0.727273  0.855978    0.797965      0.842531
recall       0.970000   0.352941  0.855978    0.661471      0.855978
f1-score     0.916535   0.475248  0.855978    0.695891      0.834993
support    300.000000  68.000000  0.855978  368.000000    368.000000


In [ ]:
svm = SVC(random_state=2)

param_grid = [
    {'C': [5, 6, 7, 8, 9, 10, 11, 12], 'kernel': ['linear']},
    {'C': [5, 6, 7, 8, 9, 10, 11, 12], 'gamma': [0.001, 0.002, 0.003, 0.004 , 0.005], 'kernel': ['rbf']}
]

search = GridSearchCV(svm, param_grid=param_grid, scoring='roc_auc', cv=3, refit=True, verbose=1)
search.fit(X_train_processed, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


GridSearchCV(cv=3, estimator=SVC(random_state=2),
             param_grid=[{'C': [5, 6, 7, 8, 9, 10, 11, 12],
                          'kernel': ['linear']},
                         {'C': [5, 6, 7, 8, 9, 10, 11, 12],
                          'gamma': [0.001, 0.002, 0.003, 0.004, 0.005],
                          'kernel': ['rbf']}],
             scoring='roc_auc', verbose=1)

In [ ]:

svm = SVC(**search.best_params_)
svm.fit(X_train_processed, y_train)

evaluate(svm, X_train_processed, X_test_processed, y_train, y_test)

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[931   2]
 [108  61]]
ACCURACY SCORE:
0.9002
CLASSIFICATION REPORT:
                   No         Yes  accuracy    macro avg  weighted avg
precision    0.896054    0.968254  0.900181     0.932154      0.907126
recall       0.997856    0.360947  0.900181     0.679402      0.900181
f1-score     0.944219    0.525862  0.900181     0.735041      0.880061
support    933.000000  169.000000  0.900181  1102.000000   1102.000000
TESTING RESULTS: 
CONFUSION MATRIX:
[[299   1]
 [ 53  15]]
ACCURACY SCORE:
0.8533
CLASSIFICATION REPORT:
                   No        Yes  accuracy   macro avg  weighted avg
precision    0.849432   0.937500  0.853261    0.893466      0.865705
recall       0.996667   0.220588  0.853261    0.608627      0.853261
f1-score     0.917178   0.357143  0.853261    0.637160      0.813693
support    300.000000  68.000000  0.853261  368.000000    368.000000
